In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def formula(num:int) -> int:

    return int(num-1)*20

def extract_cities(page) -> BeautifulSoup:
    
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
    url = f'https://www.tripadvisor.com.mx/Restaurants-g150768-oa{page}-Mexico.html#LOCATION_LIST'
    r = requests.get(url, headers=headers)

    soup = BeautifulSoup(r.content, 'html.parser')

    return soup

def transform_cities(soup) -> list:
    
    cities_ = []
    items = soup.find_all('div', class_='geo_wrap')

    for item in items:
        city =  item.find('a').text
        href =  item.find('a')['href']

        city = {'city': city, 'href': href}
        cities_.append(city)

    return cities_

def transform_cities2(soup) -> list:
    
    cities_ = []
    items = soup.find_all('ul', {'class': 'geoList'})

    for ultag in items:
        for litag in ultag.find_all('li'):
            city = litag.text
            href = litag.find('a')['href']
            
            city = {'city':city,'href':href}
            cities_.append(city)
    return cities_

In [3]:
max_page = 3

cities = transform_cities(extract_cities(0))

for i in range(20,(max_page*20),20):
    cities_ = transform_cities2(extract_cities(i))
    cities += cities_

In [4]:
cities = pd.DataFrame(cities)
len(cities)

60

In [5]:
cities['city'] = cities['city'].str.lstrip('Restaurantes en')
cities['state'] = ''

for i in range(len(cities)):
    if '\n-' in cities['city'][i]:
        cities['state'][i] = cities['city'][i].split('\n-')[1]
        cities['city'][i] = cities['city'][i].split('\n-')[0]

cities = cities[['city','state','href']]
cities.head()

,city,state,href
0,Ciudad de México,,/Restaurants-g150800-Mexico_City_Central_Mexic...
1,Guadalajara,,/Restaurants-g150798-Guadalajara_Guadalajara_M...
2,Cancún,,/Restaurants-g150807-Cancun_Yucatan_Peninsula....
3,Playa del Carmen,,/Restaurants-g150812-Playa_del_Carmen_Yucatan_...
4,Monterrey,,/Restaurants-g150782-Monterrey_Northern_Mexico...


In [6]:
def extract_restaurant_list(href_city) -> BeautifulSoup:
    
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
    url = f'https://www.tripadvisor.com.mx/{href_city}'
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    return soup

In [7]:
def transform_restaurant_list(soup) -> list:
    
    restaurant_ = []
    items = soup.find_all('div', class_='RfBGI')
    
    for item in items:
        restaurant =  item.find('a', class_='Lwqic Cj b').text
        href =  item.find('a')['href']

        restaurant = {'restaurant': restaurant, 'href': href}
        restaurant_.append(restaurant)

    return restaurant_

In [8]:
limites_cities = 2
max_page = 30 #2 

href_cities = cities[:limites_cities]['href'].str.split('-')
href_cities_list = []

for i in range(len(href_cities)):
    for page in range(0,max_page+1,30):

        href = href_cities[i][0] + '-' + href_cities[i][1] + '-oa' + str(page) + '-' + href_cities[i][2]
        href_cities_list.append(href)

href_cities_list

['/Restaurants-g150800-oa0-Mexico_City_Central_Mexico_and_Gulf_Coast.html',
 '/Restaurants-g150800-oa30-Mexico_City_Central_Mexico_and_Gulf_Coast.html',
 '/Restaurants-g150798-oa0-Guadalajara_Guadalajara_Metropolitan_Area.html',
 '/Restaurants-g150798-oa30-Guadalajara_Guadalajara_Metropolitan_Area.html']

In [9]:
max_page = 3
restaurants = []

for city in href_cities_list:
    restaurant = transform_restaurant_list(extract_restaurant_list(city))
    restaurants += restaurant
restaurants = pd.DataFrame(restaurants)

restaurants

,restaurant,href
0,San Giorgio Pizzeria,/Restaurant_Review-g150800-d18891710-Reviews-S...
1,1. Balta,/Restaurant_Review-g150800-d19338104-Reviews-B...
2,2. La Mansion Marriott Reforma,/Restaurant_Review-g150800-d2394477-Reviews-La...
3,3. Condimento Restaurant,/Restaurant_Review-g150800-d1926008-Reviews-Co...
4,4. La Distral,/Restaurant_Review-g150800-d12104177-Reviews-L...
...,...,...
129,56. La Casa de los Platos,/Restaurant_Review-g150798-d5488792-Reviews-La...
130,57. Hueso,/Restaurant_Review-g150798-d7175235-Reviews-Hu...
131,58. Boca del Cielo,/Restaurant_Review-g150798-d3687050-Reviews-Bo...
132,59. Save,/Restaurant_Review-g150798-d11668098-Reviews-S...


In [10]:
restaurants['sponsored'] = ''
for i in range(len(restaurants)):
    if '.' in restaurants['restaurant'][i]:
        restaurants['restaurant'][i] = restaurants['restaurant'][i].split('.')[1]
        restaurants['sponsored'][i] = restaurants['restaurant'][i].split('.')[0]

restaurants['sponsored'] = ~restaurants.sponsored.astype('bool')

restaurants.head()

,restaurant,href,sponsored
0,San Giorgio Pizzeria,/Restaurant_Review-g150800-d18891710-Reviews-S...,True
1,Balta,/Restaurant_Review-g150800-d19338104-Reviews-B...,False
2,La Mansion Marriott Reforma,/Restaurant_Review-g150800-d2394477-Reviews-La...,False
3,Condimento Restaurant,/Restaurant_Review-g150800-d1926008-Reviews-Co...,False
4,La Distral,/Restaurant_Review-g150800-d12104177-Reviews-L...,False


In [11]:
def extract_restaurant_info(href_restaurant) -> BeautifulSoup:
    
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
    url = f'https://www.tripadvisor.com.mx{href_restaurant}'
    r = requests.get(url, headers=headers)

    soup = BeautifulSoup(r.content, 'html.parser')

    return soup

In [14]:
restaurants.iloc[0][1]

'/Restaurant_Review-g150800-d18891710-Reviews-San_Giorgio_Pizzeria-Mexico_City_Central_Mexico_and_Gulf_Coast.html'

In [12]:
def transform_restaurant_info(soup) -> list:
    
    restaurant_info_ = []
    items = soup.find_all('div', class_='lBkqB _T')
    
    for item in items:
        name =  item.find('div', class_='acKDw w O').text
        points = ''
        reviews = item.find('span', class_='AfQtZ').text
        ranking = item.find('span', class_='DsyBj cNFrA').text
        category = item.find('span', class_='DsyBj DxyfE').text
        #adress = item.find('span', class_='AYHFM')['href'].text
        phone = item.find('span', class_='AYHFM').text
        open_hours = item.find('span', class_='mMkhr').text
        #web_page = item.find('a', class_='YnKZo Ci Wc _S C AYHFM')['href'].text

        restaurant_info = {'name':name, 'reviews':reviews, 'ranking':ranking, 'category':category, 'phone':phone, 'open_hours':open_hours}
        restaurant_info_.append(restaurant_info)

    return restaurant_info_



restaurant_info = []

for i in range(0,3):
    restaurant_info_ = transform_restaurant_info(extract_restaurant_info(restaurants.iloc[i][1]))
    restaurant_info += restaurant_info_

restaurant_info

[{'name': 'San Giorgio Pizzeria SolicitadoOpiniónGuardarCompartir ',
  'reviews': '64\xa0opiniones',
  'ranking': '#66 de 5,262 restaurantes en Ciudad de México',
  'category': '$$ - $$$ItalianaPizzeríaSaludable',
  'phone': '525612622993',
  'open_hours': ' Abierto ahora:\xa01:00\xa0p.\xa0m. - 11:00\xa0p.\xa0m.'},
 {'name': 'Balta SolicitadoOpiniónGuardarCompartir ',
  'reviews': '466\xa0opiniones',
  'ranking': '#1 de 5,262 restaurantes en Ciudad de México',
  'category': '$$ - $$$MexicanaMediterráneaEuropea',
  'phone': '+52 55 8660 0500',
  'open_hours': ' Abierto ahora:\xa006:00\xa0a.\xa0m. - 11:00\xa0p.\xa0m.'},
 {'name': 'La Mansion Marriott Reforma SolicitadoOpiniónGuardarCompartir ',
  'reviews': '988\xa0opiniones',
  'ranking': '#2 de 5,262 restaurantes en Ciudad de México',
  'category': '$$$$MexicanaParrilladaOpciones sin gluten',
  'phone': '+52 55 1102 7021',
  'open_hours': ' Abierto ahora:\xa01:00\xa0p.\xa0m. - 10:00\xa0p.\xa0m.'}]

In [15]:

def transform_restaurant_info(soup) -> list:
    
    restaurant_info_ = []
    images = soup.find_all('img', class_='kDZhm IdiaP Me')
    for img in images:
        if img.has_attr('src'):
            coordinates = (img['src'])
            print(coordinates)
            restaurant_info = {'coordinates':coordinates}
            restaurant_info_.append(restaurant_info)

    return restaurant_info_



restaurant_info = []

for i in range(0,3):
    restaurant_info_ = transform_restaurant_info(extract_restaurant_info(restaurants.iloc[i][1]))
    restaurant_info += restaurant_info_

restaurant_info

[]

In [ ]:
def transform_restaurant_info(soup) -> list:
    
    restaurant_info_ = []
    items = soup.find_all('div', class_='f e')
        
    for item in items:
        adress = item.find('span', class_='yEWoV').text
        #mail = item.find('div', {"data-test-target":"staticMapSnapshot"}).text
        mail = item.find('a', href=True)['href']
        coordinates = ''# item.find('span', src=True)['src']

        

        restaurant_info = {'adress':adress, 'mail':mail, 'coordinates':coordinates}
        restaurant_info_.append(restaurant_info)

    return restaurant_info_



restaurant_info = []

for i in range(0,3):
    restaurant_info_ = transform_restaurant_info(extract_restaurant_info(restaurants.iloc[i][1]))
    restaurant_info += restaurant_info_

restaurant_info

In [ ]:
from selenium import webdriver

In [ ]:
def extractRI(page) -> BeautifulSoup:

    #headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:103.0) Gecko/20100101 Firefox/103.0'}
    url = f'https://www.tripadvisor.com.mx{page}'
    #r = requests.get(url, headers=headers)

    browser = webdriver.Chrome(executable_path=r"/Users/rosaarzabala/Documents/Projects/TripAdvisorScrapper/chromedriver")
    browser.get(url)
    html = browser.page_source

    soup = BeautifulSoup(html , 'html.parser')

    return soup

In [ ]:
def transform_restaurant_info(soup) -> list:
    
    restaurant_info_ = []
    items = soup.find_all('div', class_='kDZhm IdiaP Me')
        
    for item in items:
        try:
            latitude = item.find('a', class_='YnKZo Ci Wc _S C FPPgD')['href']
            #print(latitude)
            restaurant_info = {'coordinates':latitude}
            restaurant_info_.append(restaurant_info)
        except:
            pass

    return restaurant_info_


restaurant_info = []

for i in range(0,3):
    restaurant_info_ = transform_restaurant_info(extractRI(restaurants.iloc[i][1]))
    restaurant_info += restaurant_info_

restaurant_info